In [16]:

"""
Módulo de funções de ativação para redes neurais.

Cada função de ativação tem:
- forward: calcula a ativação
- derivada: usada no backpropagation (veremos depois)
"""

class Ativacao:
    """"Classe base para funções de ativação. """

    def forward(self, x):
        """Aplica a função de ativação """
        raise NotImplementedError

    def derivada(self, x):
        """Calcula a derivada (para backpropagation) """
        raise NotImplementedError


In [17]:
# === Funções de ativação

class Degrau(Ativacao):
    """
    Função Degrau (Step Function).
    Retorna 1 se x >= limiar, senão 0.
    """

    def __init__(self, limiar=1.0):
        self.limiar = limiar

    def forward(self, x):
        return 1 if x >= self.limiar else 0

    def derivada(self, x):
        # Degrau não é diferenciável, mas podemos retornar 0
        return 0

class Sigmoid(Ativacao):
    """
    Função Sigmoid (Logística).
    Mapeia valores para range (0, 1).
    """

    def forward(self, x):
        return 1 / (1 + np.exp(-x))

    def derivada(self, x):
        s = self.forward(x)
        return s * (1 - s)

class ReLU(Ativacao):
    """
    Rectified Linear Unit.
    Retorna max(0, x).
    """

    def forward(self, x):
        return max(0, x)

    def derivada(self, x):
        return 1 if 0 >= 0 else 0

class Tanh(Ativacao):
    """
    Tangente Hiperbólica.
    Mapeia valores para range (-1, 1).
    """

    def forward(self, x):
        return np.tanh(x)

    def derivada(self, x):
        return 1 - np.tanh(x)**2

class Linear(Ativacao):

    def forward(self, x):
        return x

    def derivada(self, x):
        s = self.foward(x)
        return s



In [18]:
"""
Implementação de um neurônio artificial
"""

# from activations import Ativacao, Degrau

class Neuronio:
    """
    Um neurônio artificial com aprendizado supervisionado.

    Attributes:
        pesos (np.ndarray): Pesos sinápticos
        bias (float): Termo de viés
        taxa_aprendizado (float): Taxa de aprendizado
        ativacao (Ativacao): Função de ativação
    """

    def __init__(self, num_entradas, taxa_aprendizado=0.1, ativacao=None):

        """
        Inicializa o neurônio.

        Args:
            num_entradas (int): Número de entradas
            taxa_aprendizado (float): Taxa de aprendizado
            ativacao (Ativacao): Função de ativação (padrão: Degrau)
        """
        self.pesos = np.random.rand(num_entradas)
        self.bias = 0.0
        self.taxa_aprendizado = taxa_aprendizado

        # Se não passar ativação, usa Degrau por padrão
        self.ativacao = ativacao if ativacao is not None else Degrau()


    def forward(self, entrada):
        """
        Propagação para frente.

        Args:
            entrada: Vetor de entrada

        Returns:
            tuple: (saida_bruta, saida_ativada)
        """

        # Combinação linear
        saida_bruta = np.dot(entrada, self.pesos) + self.bias

        # Aplica ativação (agora vem do objeto de ativação)
        saida_ativada = self.ativacao.forward(saida_bruta)

        return saida_bruta, saida_ativada

    def treinar(self, entrada, esperado):
        """
        Treina o neurônio com um exemplo.

        Args:
            entrada: Vetor de entrada
            esperado: Saída esperada

        Returns:
            tuple: (saida, erro)
        """
        saida_bruta, saida = self.forward(entrada)
        erro = esperado - saida

        # Atualiza pesos
        self.pesos += self.taxa_aprendizado * erro * np.array(entrada)
        self.bias += self.taxa_aprendizado * erro

        return saida, erro

    def __repr__(self):
        """Representação em string do neurônio."""
        return f"Neuronio(entradas={len(self.pesos)}, ativacao={self.ativacao.__class__.__name__})"




In [19]:
"""
Implementação de uma camada de neurônios.
"""

import numpy as np

class Camada:
    """
    Uma camada (layer) de neurônios.

    Todos os neurônios da camada:
    - Recebem a mesma entrada
    - Podem ter ativações diferentes (ou mesma)
    - Produzem um vetor de saídas

    Attributes:
        neuronios (list): Lista de objetos Neuronio
        num_neuronios (int): Quantidade de neurônios na camada
        num_entradas (int): Número de entradas que cada neurônio recebe
    """

    def __init__(self, num_entradas, num_neuronios, taxa_aprendizado=0.1, ativacao=None):
        """
        Inicializa a camada.

        Args:
            num_entradas (int): Número de entradas para cada neurônio
            num_neuronios (int): Quantidade de neurônios na camada
            taxa_aprendizado (float): Taxa de aprendizado
            ativacao (Ativacao): Função de ativação (padrão: Degrau)
        """
        self.num_neuronios = num_neuronios
        self.num_entradas = num_entradas

        # Cria uma lista de neurônios
        self.neuronios = []
        for _ in range(num_neuronios):
            neuronio = Neuronio(
                num_entradas=num_entradas,
                taxa_aprendizado=taxa_aprendizado,
                ativacao=ativacao if ativacao is not None else Degrau()
            )
            self.neuronios.append(neuronio)

    def forward(self, entrada):
        """
        Propagação para frente através da camada.

        Args:
            entrada: Vetor de entrada (mesmo para todos os neurônios)

        Returns:
            tuple: (saidas_brutas, saidas_ativadas)
                - saidas_brutas: lista com saídas antes da ativação
                - saidas_ativadas: lista com saídas após ativação
        """
        saidas_brutas = []
        saidas_ativadas = []

        # Cada neurônio processa a mesma entrada
        for neuronio in self.neuronios:
            bruta, ativada = neuronio.forward(entrada)
            saidas_brutas.append(bruta)
            saidas_ativadas.append(ativada)

        return np.array(saidas_brutas), np.array(saidas_ativadas)

    def treinar(self, entrada, esperados):
        """
        Treina todos os neurônios da camada.

        Args:
            entrada: Vetor de entrada
            esperados: Lista de saídas esperadas (uma para cada neurônio)

        Returns:
            tuple: (saidas, erros)
        """
        if len(esperados) != self.num_neuronios:
            raise ValueError(
                f"esperados deve ter {self.num_neuronios} valores, "
                f"mas recebeu {len(esperados)}"
            )

        saidas = []
        erros = []

        # Treina cada neurônio com sua saída esperada
        for neuronio, esperado in zip(self.neuronios, esperados):
            saida, erro = neuronio.treinar(entrada, esperado)
            saidas.append(saida)
            erros.append(erro)

        return np.array(saidas), np.array(erros)

    def get_pesos(self):
        """
        Retorna os pesos de todos os neurônios como matriz.

        Returns:
            np.ndarray: Matriz de pesos (num_neuronios x num_entradas)
        """
        pesos_matriz = []
        for neuronio in self.neuronios:
            pesos_matriz.append(neuronio.pesos)
        return np.array(pesos_matriz)

    def get_bias(self):
        """
        Retorna os bias de todos os neurônios.

        Returns:
            np.ndarray: Vetor de bias
        """
        return np.array([neuronio.bias for neuronio in self.neuronios])

    def __repr__(self):
        """Representação em string da camada."""
        ativacao_nome = self.neuronios[0].ativacao.__class__.__name__
        return (
            f"Camada(neuronios={self.num_neuronios}, "
            f"entradas={self.num_entradas}, "
            f"ativacao={ativacao_nome})"
        )

    def __len__(self):
        """Retorna o número de neurônios."""
        return self.num_neuronios


class CamadaVetorizada:
    """
    Versão OTIMIZADA da camada usando operações vetorizadas.

    Muito mais rápida que criar neurônios individuais!
    Essa é a abordagem usada em bibliotecas profissionais.
    """

    def __init__(self, num_entradas, num_neuronios, taxa_aprendizado=0.1, ativacao=None):
        """
        Inicializa a camada vetorizada.

        Args:
            num_entradas (int): Dimensão da entrada
            num_neuronios (int): Número de neurônios (dimensão da saída)
            taxa_aprendizado (float): Taxa de aprendizado
            ativacao (Ativacao): Função de ativação
        """
        self.num_neuronios = num_neuronios
        self.num_entradas = num_entradas
        self.taxa_aprendizado = taxa_aprendizado
        self.ativacao = ativacao if ativacao is not None else Degrau()

        # Matriz de pesos: (num_neuronios x num_entradas)
        # Cada LINHA é um neurônio
        self.pesos = np.random.rand(num_neuronios, num_entradas)

        # Vetor de bias: (num_neuronios,)
        self.bias = np.zeros(num_neuronios)

    def forward(self, entrada):
        """
        Propagação vetorizada (MUITO mais rápida!).

        Args:
            entrada: Vetor de entrada (num_entradas,)

        Returns:
            tuple: (saidas_brutas, saidas_ativadas)
        """
        # Multiplicação matricial: (num_neuronios x num_entradas) @ (num_entradas,)
        # Resultado: (num_neuronios,)
        saidas_brutas = self.pesos @ entrada + self.bias

        # Aplica ativação em cada elemento
        saidas_ativadas = np.array([
            self.ativacao.forward(x) for x in saidas_brutas
        ])

        return saidas_brutas, saidas_ativadas

    def treinar(self, entrada, esperados):
        """
        Treinamento vetorizado.

        Args:
            entrada: Vetor de entrada
            esperados: Vetor de saídas esperadas

        Returns:
            tuple: (saidas, erros)
        """
        saidas_brutas, saidas = self.forward(entrada)
        erros = esperados - saidas

        # Atualização vetorizada dos pesos
        # Cada linha (neurônio) é atualizada: peso += taxa * erro * entrada
        entrada_array = np.array(entrada)
        for i in range(self.num_neuronios):
            self.pesos[i] += self.taxa_aprendizado * erros[i] * entrada_array
            self.bias[i] += self.taxa_aprendizado * erros[i]

        return saidas, erros

    def __repr__(self):
        return (
            f"CamadaVetorizada(neuronios={self.num_neuronios}, "
            f"entradas={self.num_entradas}, "
            f"ativacao={self.ativacao.__class__.__name__})"
        )

In [20]:
"""
Testes para a classe Camada.
"""

import numpy as np


print("=" * 60)
print("TESTE 1: Camada com 3 neurônios")
print("=" * 60)

# Cria uma camada: 10 entradas → 3 neurônios
camada = Camada(
    num_entradas=10,
    num_neuronios=3,
    taxa_aprendizado=0.1,
    ativacao=Sigmoid()
)

print(f"\n{camada}")
print(f"Número de neurônios: {len(camada)}")

# Entrada de teste
entrada = [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]

# Saídas esperadas para cada neurônio
esperados = [1, 0, 1]  # Neurônio 1: quer 1, Neurônio 2: quer 0, Neurônio 3: quer 1

print("\n--- Treinamento ---")
for epoca in range(10):
    saidas, erros = camada.treinar(entrada, esperados)
    print(f"Época {epoca} | saídas={saidas} | erros={erros}")

# Testa após treinamento
print("\n--- Teste Final ---")
_, saidas_finais = camada.forward(entrada)
print(f"Entrada: {entrada}")
print(f"Saídas: {saidas_finais}")
print(f"Esperado: {esperados}")


print("\n" + "=" * 60)
print("TESTE 2: Comparando Camada vs CamadaVetorizada")
print("=" * 60)

# Mesma configuração para ambas
entrada = np.array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
esperados = np.array([1, 0, 1])

# Camada normal
camada_normal = Camada(10, 3, taxa_aprendizado=0.1, ativacao=Sigmoid())

# Camada vetorizada
camada_vet = CamadaVetorizada(10, 3, taxa_aprendizado=0.1, ativacao=Sigmoid())

print("\nTreinando ambas por 100 épocas...")

# Treina camada normal
for _ in range(100):
    camada_normal.treinar(entrada, esperados)

# Treina camada vetorizada
for _ in range(100):
    camada_vet.treinar(entrada, esperados)

# Compara resultados
_, saidas_normal = camada_normal.forward(entrada)
_, saidas_vet = camada_vet.forward(entrada)

print(f"\nCamada Normal: {saidas_normal}")
print(f"Camada Vetorizada: {saidas_vet}")
print(f"Esperado: {esperados}")


print("\n" + "=" * 60)
print("TESTE 3: Visualizando Pesos da Camada")
print("=" * 60)

camada = Camada(5, 2, ativacao=ReLU())

print(f"\nPesos iniciais:")
print(camada.get_pesos())
print(f"\nBias iniciais:")
print(camada.get_bias())

# Treina um pouco
entrada = [1, 0, 1, 0, 1]
esperados = [1, 0]

for _ in range(50):
    camada.treinar(entrada, esperados)

print(f"\nPesos após treinamento:")
print(camada.get_pesos())
print(f"\nBias após treinamento:")
print(camada.get_bias())

TESTE 1: Camada com 3 neurônios

Camada(neuronios=3, entradas=10, ativacao=Sigmoid)
Número de neurônios: 3

--- Treinamento ---
Época 0 | saídas=[0.53505645 0.58097709 0.62260904] | erros=[ 0.46494355 -0.58097709  0.37739096]
Época 1 | saídas=[0.55809764 0.55245387 0.64017305] | erros=[ 0.44190236 -0.55245387  0.35982695]
Época 2 | saídas=[0.57976906 0.52500401 0.65657782] | erros=[ 0.42023094 -0.52500401  0.34342218]
Época 3 | saídas=[0.60009756 0.49877469 0.67189428] | erros=[ 0.39990244 -0.49877469  0.32810572]
Época 4 | saídas=[0.61912881 0.47385981 0.68619418] | erros=[ 0.38087119 -0.47385981  0.31380582]
Época 5 | saídas=[0.63692127 0.45030755 0.69954822] | erros=[ 0.36307873 -0.45030755  0.30045178]
Época 6 | saídas=[0.6535413  0.42812881 0.71202468] | erros=[ 0.3464587  -0.42812881  0.28797532]
Época 7 | saídas=[0.66905938 0.40730554 0.72368863] | erros=[ 0.33094062 -0.40730554  0.27631137]
Época 8 | saídas=[0.68354728 0.38779828 0.73460139] | erros=[ 0.31645272 -0.38779828  0.

### Visualização do TESTE 1: Parâmetros da Camada (Antes e Depois do Treinamento)

In [21]:
"""
Implementação de uma rede neural feedforward.
"""


class RedeNeural:
    """
    Rede neural feedforward (camadas totalmente conectadas).

    A saída de uma camada vira entrada da próxima.

    Attributes:
        camadas (list): Lista de objetos CamadaVetorizada
        historico (dict): Histórico de treinamento (loss, acurácia)
    """

    def __init__(self, arquitetura, taxa_aprendizado=0.1, ativacao=None):
        """
        Inicializa a rede neural.

        Args:
            arquitetura (list): Lista com número de neurônios por camada
                Exemplo: [10, 5, 3, 2] significa:
                - Entrada: 10 features
                - Hidden 1: 5 neurônios
                - Hidden 2: 3 neurônios
                - Saída: 2 neurônios
            taxa_aprendizado (float): Taxa de aprendizado
            ativacao (Ativacao): Função de ativação (padrão: Sigmoid)
        """
        if len(arquitetura) < 2:
            raise ValueError("Arquitetura deve ter pelo menos 2 camadas (entrada + saída)")

        self.arquitetura = arquitetura
        self.camadas = []
        self.historico = {
            'loss': [],
            'acuracia': []
        }

        # Cria as camadas
        for i in range(len(arquitetura) - 1):
            num_entradas = arquitetura[i]
            num_neuronios = arquitetura[i + 1]

            camada = CamadaVetorizada(
                num_entradas=num_entradas,
                num_neuronios=num_neuronios,
                taxa_aprendizado=taxa_aprendizado,
                ativacao=ativacao if ativacao is not None else Sigmoid()
            )
            self.camadas.append(camada)

    def forward(self, entrada):
        """
        Propagação para frente através de todas as camadas.

        Args:
            entrada: Vetor/matriz de entrada

        Returns:
            tuple: (todas_ativacoes, saida_final)
                - todas_ativacoes: lista com saídas de cada camada
                - saida_final: saída da última camada
        """
        ativacoes = [np.array(entrada)]  # Começa com a entrada

        # Propaga através de cada camada
        for camada in self.camadas:
            _, saida_ativada = camada.forward(ativacoes[-1])
            ativacoes.append(saida_ativada)

        return ativacoes, ativacoes[-1]

    def treinar_epoca(self, entradas, esperados):
        """
        Treina a rede por uma época (todos os exemplos).

        Args:
            entradas (list): Lista de vetores de entrada
            esperados (list): Lista de vetores de saída esperada

        Returns:
            dict: Métricas da época (loss médio, acurácia)
        """
        total_loss = 0
        acertos = 0

        for entrada, esperado in zip(entradas, esperados):
            # Forward pass
            todas_ativacoes, saida = self.forward(entrada)

            # Calcula erro/loss
            erro = np.array(esperado) - saida
            loss = np.mean(erro ** 2)  # MSE (Mean Squared Error)
            total_loss += loss

            # Calcula acurácia (para classificação)
            predicao = np.argmax(saida) if len(saida) > 1 else round(saida[0])
            alvo = np.argmax(esperado) if len(esperado) > 1 else esperado[0]
            if predicao == alvo:
                acertos += 1

            # Backward pass (simplificado - só última camada por enquanto)
            self._treinar_ultima_camada(todas_ativacoes[-2], esperado)

        # Métricas da época
        num_exemplos = len(entradas)
        metricas = {
            'loss': total_loss / num_exemplos,
            'acuracia': acertos / num_exemplos
        }

        return metricas

    def _treinar_ultima_camada(self, entrada_camada, esperado):
        """
        Treina apenas a última camada (simplificação).

        Args:
            entrada_camada: Entrada que chegou na última camada
            esperado: Saída esperada
        """
        self.camadas[-1].treinar(entrada_camada, esperado)

    def treinar(self, entradas, esperados, num_epocas=100, verbose=True):
        """
        Treina a rede por múltiplas épocas.

        Args:
            entradas (list): Conjunto de entradas
            esperados (list): Conjunto de saídas esperadas
            num_epocas (int): Número de épocas
            verbose (bool): Mostrar progresso

        Returns:
            dict: Histórico completo de treinamento
        """
        for epoca in range(num_epocas):
            metricas = self.treinar_epoca(entradas, esperados)

            # Salva no histórico
            self.historico['loss'].append(metricas['loss'])
            self.historico['acuracia'].append(metricas['acuracia'])

            # Mostra progresso
            if verbose and (epoca % 10 == 0 or epoca == num_epocas - 1):
                print(f"Época {epoca:3d} | "
                      f"Loss: {metricas['loss']:.4f} | "
                      f"Acurácia: {metricas['acuracia']:.2%}")

        return self.historico

    def prever(self, entrada):
        """
        Faz predição para uma entrada.

        Args:
            entrada: Vetor de entrada

        Returns:
            np.ndarray: Saída da rede
        """
        _, saida = self.forward(entrada)
        return saida

    def __repr__(self):
        camadas_str = " → ".join(map(str, self.arquitetura))
        return f"RedeNeural({camadas_str})"

In [22]:
"""
Testes completos para a rede neural com validação de aprendizado.
"""

print("=" * 70)
print("TESTE 1: Rede Neural para Porta Lógica OR")
print("=" * 70)

# Dataset: Porta OR
# 0 OR 0 = 0
# 0 OR 1 = 1
# 1 OR 0 = 1
# 1 OR 1 = 1

entradas_treino = [
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
]

saidas_treino = [
    [0],  # 0 OR 0 = 0
    [1],  # 0 OR 1 = 1
    [1],  # 1 OR 0 = 1
    [1]   # 1 OR 1 = 1
]

# Cria rede: 2 entradas → 3 neurônios hidden → 1 saída
rede = RedeNeural(
    arquitetura=[2, 3, 1],
    taxa_aprendizado=0.5,
    ativacao=Sigmoid()
)

print(f"\n{rede}")
print(f"Número de camadas: {len(rede.camadas)}")

# Treina
print("\n--- Treinamento ---")
rede.treinar(entradas_treino, saidas_treino, num_epocas=100, verbose=True)

# Testa com dados de treinamento
print("\n--- Validação com Dados de Treinamento ---")
for entrada, esperado in zip(entradas_treino, saidas_treino):
    predicao = rede.prever(entrada)
    print(f"Entrada: {entrada} | Esperado: {esperado[0]} | "
          f"Predição: {predicao[0]:.4f} | "
          f"Correto: {'✓' if abs(predicao[0] - esperado[0]) < 0.5 else '✗'}")

# Testa com novas entradas (mesmas, mas verificando consistência)
print("\n--- Teste de Consistência (rodando predições novamente) ---")
for entrada, esperado in zip(entradas_treino, saidas_treino):
    predicao = rede.prever(entrada)
    print(f"Entrada: {entrada} | Predição: {predicao[0]:.4f}")


print("\n" + "=" * 70)
print("TESTE 2: Rede Neural para Porta Lógica XOR (mais difícil!)")
print("=" * 70)

# Dataset: Porta XOR (não é linearmente separável!)
# 0 XOR 0 = 0
# 0 XOR 1 = 1
# 1 XOR 0 = 1
# 1 XOR 1 = 0

entradas_xor = [
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
]

saidas_xor = [
    [0],  # 0 XOR 0 = 0
    [1],  # 0 XOR 1 = 1
    [1],  # 1 XOR 0 = 1
    [0]   # 1 XOR 1 = 0
]

# Rede maior para XOR
rede_xor = RedeNeural(
    arquitetura=[2, 4, 1],  # Precisa de hidden layer maior
    taxa_aprendizado=0.3,
    ativacao=Sigmoid()
)

print(f"\n{rede_xor}")

print("\n--- Treinamento ---")
rede_xor.treinar(entradas_xor, saidas_xor, num_epocas=500, verbose=True)

print("\n--- Validação XOR ---")
for entrada, esperado in zip(entradas_xor, saidas_xor):
    predicao = rede_xor.prever(entrada)
    resultado = "✓" if abs(predicao[0] - esperado[0]) < 0.5 else "✗"
    print(f"Entrada: {entrada} | Esperado: {esperado[0]} | "
          f"Predição: {predicao[0]:.4f} | {resultado}")


print("\n" + "=" * 70)
print("TESTE 3: Classificação Multi-Classe (3 classes)")
print("=" * 70)

# Dataset sintético: 3 padrões diferentes
# Padrão A: [1, 0, 0, ...] → Classe 0
# Padrão B: [0, 1, 0, ...] → Classe 1
# Padrão C: [0, 0, 1, ...] → Classe 2

entradas_multi = [
    # Classe 0
    [1, 0, 0, 0, 0],
    [1, 1, 0, 0, 0],
    [1, 0, 1, 0, 0],
    # Classe 1
    [0, 1, 0, 0, 0],
    [0, 1, 1, 0, 0],
    [0, 1, 0, 1, 0],
    # Classe 2
    [0, 0, 1, 0, 0],
    [0, 0, 1, 1, 0],
    [0, 0, 1, 0, 1],
]

saidas_multi = [
    # Classe 0 (one-hot encoding)
    [1, 0, 0],
    [1, 0, 0],
    [1, 0, 0],
    # Classe 1
    [0, 1, 0],
    [0, 1, 0],
    [0, 1, 0],
    # Classe 2
    [0, 0, 1],
    [0, 0, 1],
    [0, 0, 1],
]

# Rede: 5 entradas → 8 hidden → 3 saídas
rede_multi = RedeNeural(
    arquitetura=[5, 8, 3],
    taxa_aprendizado=0.2,
    ativacao=Sigmoid()
)

print(f"\n{rede_multi}")

print("\n--- Treinamento ---")
rede_multi.treinar(entradas_multi, saidas_multi, num_epocas=200, verbose=True)

print("\n--- Validação Multi-Classe ---")
for i, (entrada, esperado) in enumerate(zip(entradas_multi, saidas_multi)):
    predicao = rede_multi.prever(entrada)
    classe_esperada = np.argmax(esperado)
    classe_predita = np.argmax(predicao)
    correto = "✓" if classe_esperada == classe_predita else "✗"

    print(f"Exemplo {i+1} | Entrada: {entrada}")
    print(f"  Esperado: Classe {classe_esperada} {esperado}")
    print(f"  Predito:  Classe {classe_predita} {predicao.round(3)} {correto}")

# Testa com dados NOVOS (variações)
print("\n--- Teste com Dados NOVOS (Generalização) ---")
entradas_teste = [
    [1, 1, 1, 0, 0],  # Deve prever Classe 0 (começa com 1)
    [0, 1, 1, 1, 0],  # Deve prever Classe 1 (tem 1 na pos 1)
    [0, 0, 1, 1, 1],  # Deve prever Classe 2 (tem 1 na pos 2)
]

for i, entrada in enumerate(entradas_teste):
    predicao = rede_multi.prever(entrada)
    classe_predita = np.argmax(predicao)
    print(f"Entrada NOVA {i+1}: {entrada}")
    print(f"  Predição: Classe {classe_predita} {predicao.round(3)}")


print("\n" + "=" * 70)
print("TESTE 4: Visualização do Histórico de Treinamento")
print("=" * 70)

print("\n--- Evolução do Loss (OR) ---")
historico_or = rede.historico
for i in [0, 25, 50, 75, 99]:
    if i < len(historico_or['loss']):
        print(f"Época {i:3d}: Loss = {historico_or['loss'][i]:.6f}")

print("\n--- Evolução da Acurácia (XOR) ---")
historico_xor = rede_xor.historico
epocas_mostrar = [0, 100, 200, 300, 400, 499]
for i in epocas_mostrar:
    if i < len(historico_xor['acuracia']):
        print(f"Época {i:3d}: Acurácia = {historico_xor['acuracia'][i]:.2%}")

TESTE 1: Rede Neural para Porta Lógica OR

RedeNeural(2 → 3 → 1)
Número de camadas: 2

--- Treinamento ---
Época   0 | Loss: 0.2247 | Acurácia: 50.00%
Época  10 | Loss: 0.2111 | Acurácia: 75.00%
Época  20 | Loss: 0.2088 | Acurácia: 75.00%
Época  30 | Loss: 0.2065 | Acurácia: 75.00%
Época  40 | Loss: 0.2043 | Acurácia: 75.00%
Época  50 | Loss: 0.2020 | Acurácia: 75.00%
Época  60 | Loss: 0.1998 | Acurácia: 75.00%
Época  70 | Loss: 0.1976 | Acurácia: 75.00%
Época  80 | Loss: 0.1954 | Acurácia: 75.00%
Época  90 | Loss: 0.1932 | Acurácia: 75.00%
Época  99 | Loss: 0.1912 | Acurácia: 75.00%

--- Validação com Dados de Treinamento ---
Entrada: [0, 0] | Esperado: 0 | Predição: 0.7612 | Correto: ✗
Entrada: [0, 1] | Esperado: 1 | Predição: 0.8124 | Correto: ✓
Entrada: [1, 0] | Esperado: 1 | Predição: 0.8219 | Correto: ✓
Entrada: [1, 1] | Esperado: 1 | Predição: 0.8609 | Correto: ✓

--- Teste de Consistência (rodando predições novamente) ---
Entrada: [0, 0] | Predição: 0.7612
Entrada: [0, 1] | Pre